## このNotebookファイルを複製（本ファイルは書込み禁止）

In [11]:
import shutil, os
dst = shutil.copy('Sample.ipynb', 'User.ipynb')
os.chmod(dst, 0o755)
print("> " + dst)

> User.ipynb


## 追加ライブラリのインストール（無ければ）

In [ ]:
!pip list | grep pandas > /dev/null; if [ $? -ne 0 ]; then pip install pandas; fi
!pip install -U pandas numpy
#!apt list --installed | grep "tf-transformations" > /dev/null; if [ $? -ne 0 ]; then apt update && apt install ros-foxy-tf-transformations && source /opt/ros/foxy/local_setup.sh ; fi
#!source /opt/ros/foxy/local_setup.sh

## 経由地点リストの初期化

In [ ]:
import pandas as pd
import numpy as np
import rclpy
rclpy.init()
waypoint_list = pd.DataFrame(index=[], columns=["x","y","deg","v_xy","v_w","tx","ty","tr"])
waypoint_list = pd.DataFrame([[0, 0, 0, 0.5, 3.14, np.inf, np.inf, np.inf]], columns=["x","y","deg","v_xy","v_w","tx","ty","tr"]) # Debug

## "リモコンでロボットを経由地点まで動かし下のCellを実行"を経由地点の数に応じて繰り返す

In [ ]:
from rclpy.node import Node
from scipy.spatial.transform import Rotation as R
from geometry_msgs.msg import TransformStamped
import tf_transformations

def quaternion_to_euler_zyx(q):
    r = R.from_quat([q[0], q[1], q[2], q[3]])
    return r.as_euler('zyx', degrees=True)
    
class SampleNodeClient(Node):
    def __init__(self):
        super().__init__('sample_node_notebook')
        self.timer = self.create_timer(5.0, self.callback_timer)
        self.subs = [self.create_subscription(TransformStamped, '/vslam/rig_tf', self.callback_rig_tf, 10)]

    def destroy(self):
        _ = [self.destroy_subscription(_sub) for _sub in self.subs]
        self.destroy_node()
    
    def callback_timer(self):
        print("waiting for the massage")

    def callback_rig_tf(self, msg):
        print(str(msg))
        _x, _y = msg.transform.translation.x, msg.transform.translation.x
        _roll, _pitch, _yaw = tf_transformations.euler_from_quaternion(
            [msg.transform.rotation.x, msg.transform.rotation.y, msg.transform.rotation.z, msg.transform.rotation.w])
        _df = pd.DataFrame([[_x, _y, np.rad2deg(_yaw), 0.5, 3.14, 0.1, 0.1, 1.0]], columns=["x","y","deg","v_xy","v_w","tx","ty","tr"])
        waypoint_list = pd.concat([waypoint_list, _df]).reset_index(drop=True)
        self.destroy()
        
node = SampleNodeClient()
try:
    rclpy.spin(node)
except KeyboardInterrupt:
    print("KeyboardInterrupt")
node.destroy()
if waypoint_list.shape[0]:
    print(waypoint_list)

## 速度や目標精度を手で修正

In [ ]:
# 経由地点index 0までの水平移動速度を0.3m/sに設定する
waypoint_list.loc[0, 'v_xy'] = 0.3
# 経由地点index 0までの旋回速度を1.57rad/sに設定する
waypoint_list.loc[0, 'v_w'] = 1.57
# 経由地点index 0の目標位置精度を±0.05mに設定する
waypoint_list.loc[0, ['tx','ty']] = 0.05
# 経由地点index 0の目標角度精度を±5degに設定する
waypoint_list.loc[0, 'tr'] = 5.0
# 経由地点一覧を表示
print(waypoint_list)

## 自律移動を実行する

In [ ]:
import copy
from std_msgs.msg import Empty
try:
    from triorb_drive_interface.msg import TriorbSetPos3 as TriorbSetPos3Topic
    from triorb_drive_interface.msg import TriorbRunResult
    from triorb_drive_interface.msg import DriveGains
except: # OLD
    from triorb_msgs.msg import TriorbSetPos3 as TriorbSetPos3Topic
    from triorb_msgs.msg import TriorbRunResult
    from triorb_msgs.msg import TriorbPID as DriveGains

STD_ACC = 1000
STD_DEC = 1000
GAIN_XY_P = 0.20
GAIN_XY_I = 0.0
GAIN_XY_D = 1e-5
GAIN_W_P = 0.20
GAIN_W_I = 0.0
GAIN_W_D = 1e-6

class SampleNavigateClient(Node):
    def __init__(self, waypoints):
        super().__init__('sample_navigate_notebook')
        self.pub_gain = self.create_publisher(DriveGains, "setting/drive/gains", 10)
        self.pub_set_pos = self.create_publisher(TriorbSetPos3Topic, "drive/set_pos", 10)
        self.pub_stop = self.create_publisher(Empty, "drive/stop", 10)
        self.subs = [self.create_subscription(TriorbRunResult, 'drive/result', self.callback_result, 10)]
        self.waypoints = copy.deepcopy(waypoints)
        self.set_gain()
        self.run_next()

    def set_gain(self):
        msg = DriveGains()
        msg.xy_p = GAIN_XY_P
        msg.xy_i = GAIN_XY_I
        msg.xy_d = GAIN_XY_D
        msg.w_p = GAIN_W_P
        msg.w_i = GAIN_W_I
        msg.w_d = GAIN_W_D
        print(str(msg))
        self.pub_gain.publish(msg)

    def run_next(self):
        for idx, point in self.waypoints.iterrows():
            request = TriorbSetPos3Topic()
            request.setting = copy.deepcopy(self.setting)
            request.pos.position.x = point.x
            request.pos.position.y = point.y
            request.pos.position.deg = point.deg
            request.pos.speed.xy = point.v_xy
            request.pos.speed.w = point.v_w
            request.pos.speed.acc = STD_ACC
            request.pos.speed.dec = STD_DEC
            request.setting.tx = point.tx
            request.setting.ty = point.ty
            request.setting.tr = point.tr
            request.setting.force = 1
            print(str(request))
            self.pub_set_pos.publish(request)
            self.waypoints = self.waypoints.drop(idx, axis=0)
            return
        self.destroy()

    def callback_result(self, msg):
        print("Receive results")
        print(str(msg))
    
    def destroy(self):
        self.pub_stop.publish(Empty())
        _ = [self.destroy_subscription(_sub) for _sub in self.subs]
        self.destroy_node()
        self.run_next()

node = SampleNavigateClient()
try:
    rclpy.spin(node)
except KeyboardInterrupt:
    print("KeyboardInterrupt")
node.destroy()